#### **Ingesta del archivo "person.json"**

In [0]:
dbutils.widgets.text("environment","")
var_environment = dbutils.widgets.get("environment")

In [0]:
dbutils.widgets.text("file_date","2024-12-30")
var_file_date = dbutils.widgets.get("file_date")

In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/common_functions"

##### Librerías

In [0]:
from pyspark.sql.functions import current_timestamp,lit,col,concat
from pyspark.sql.types import StructType,StructField,StringType,IntegerType

##### Definir esquema

In [0]:
name_schema = StructType([
    StructField("forename",StringType(),True),
    StructField("surname",StringType(),True)
])

In [0]:
person_schema = StructType([
    StructField("personId",IntegerType(),False),
    StructField("personName",name_schema)
])

##### Leer JSON

In [0]:
df_person = spark.read.schema(person_schema).json(f"{bronze_folder_path}/{var_file_date}/person.json")

##### Renombrar y añadir columnas

In [0]:
df_person_with_columns = add_ingestion_date(df_person).withColumnRenamed("personId","person_id")\
                                  .withColumn("environment",lit(var_environment)) \
                                  .withColumn("name",concat(col("personName.foreName"),lit(" "),col("personName.surName"))).withColumn("file_date",lit(var_file_date))

In [0]:
df_person_final = df_person_with_columns.drop(df_person_with_columns.personName)

##### Escribir dataframe en Silver

In [0]:
merge_condition = "target.person_id = source.person_id AND target.file_date = source.file_date"
merge_delta_lake(df_person_final,"movie_silver","persons",silver_folder_path,merge_condition,"file_date")

In [0]:
%sql
SELECT COUNT(1) as TOTAL,file_date
FROM movie_silver.persons
GROUP BY file_date

In [0]:
dbutils.notebook.exit("Success")